In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2 as pg
import numpy as np
import os
from tqdm import tqdm

In [ ]:
csv_path =  os.path.abspath(".")[:-16]+"Data\\"
myData = pd.read_csv(+csv_path"Updated_Data_2.csv",header=0,index_col=[0],low_memory=False)
myData.head()

<h4>Detail about our Data and optimize the reading process for quick interactions</h4>

In [ ]:
myData.info() #first look

In [ ]:
myData.info(memory_usage='deep') #deep look

In [ ]:
for c in myData:
    k=myData[str(c)].unique()
    if len(k)<60 and str(c): #our data is composed of 15 million rows (category of 60 element is still saving memory usage)
        myData[str(c)]=myData[str(c)].astype("category")

In [ ]:
myData.info(memory_usage='deep') #result